In [1]:
!apt-get update && apt-get upgrade -y
!apt install -y libgl1-mesa-dev
!apt install -y libglib2.0-0

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 2s (164 kB/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libnvinfer-plugin7 libnvinfer7
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Readin

In [2]:
!pip install opencv-python
!pip install tqdm
!pip install scikit-learn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!nvidia-smi

Mon Dec 12 03:31:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 33%   30C    P8    23W / 340W |     17MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from IPython.display import Image, display

In [5]:
from matplotlib import pyplot as plt

In [6]:
import cv2
from PIL import ImageFile
from PIL import Image

In [7]:
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [9]:
import random
import tqdm
import glob

In [10]:
from datetime import datetime

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
import sys
import pathlib
import os
import shutil
import math
import time

In [13]:
import numpy as np

In [14]:
window_width = 256
window_height = 256
model_name="00/scannoise_remover"
testimage_epoch = 11
testimages_max = 30
#random.seed(time.time())
random.seed(6)
validation_percent = 0.1

In [15]:
# split noised data to training and test (validation) sets
filenames_train_list=sorted(glob.glob('/work/noised/*/*.png'))
X_train, x_test= train_test_split(filenames_train_list, test_size=validation_percent, random_state=42) #from sklearn
Y_train = [s.replace('noised', 'clean') for s in X_train]
random.shuffle(x_test)
y_test = [s.replace('noised', 'clean') for s in x_test]

In [16]:
X_test = np.array([np.array(Image.open(file)) for file in x_test[0:testimages_max]])
Y_test = np.array([np.array(Image.open(file)) for file in y_test[0:testimages_max]])
X_test = X_test.astype('float32')
X_test /= 255
Y_test = Y_test.astype('float32')
Y_test /= 255

In [17]:
# definition to show original image and reconstructed image
def showOrigDec(orig, noise, denoise, num=5):
    import matplotlib.pyplot as plt
    n = num
    m = 6
    plt.figure(dpi=600)  #figsize=(20, 6))

    for i in range(n):
        # display original
        ax = plt.subplot(m, n, i+1)
        _orig = np.copy(orig[i])
        _orig *= 255
        _orig = _orig.astype('uint8')
        plt.imshow(_orig.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # sobel of clean image
        ax = plt.subplot(m, n, i + 1 + n)
        kernel = np.ones((5,5),np.float32)/25
        __orig = cv2.filter2D(_orig,-1,kernel)
        sobelx64f = cv2.Sobel(__orig,cv2.CV_64F,1,0,ksize=5)
        abs_sobel64f = np.absolute(sobelx64f)
        sobel_8u = np.uint8(abs_sobel64f)
        plt.imshow(sobel_8u.reshape(window_height, window_width, 3)) 
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)        
        
        
        # display noisy image
        ax = plt.subplot(m, n, i + 1 + 2*n)
        _noise = np.copy(noise[i])
        _noise *= 255
        _noise = _noise.astype('uint8')
        plt.imshow(_noise.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # sobel of noisy image
        ax = plt.subplot(m, n, i + 1 + 3*n)
        kernel = np.ones((5,5),np.float32)/25
        __noise = cv2.filter2D(_noise,-1,kernel)
        sobelx64f = cv2.Sobel(__noise,cv2.CV_64F,1,0,ksize=5)
        abs_sobel64f = np.absolute(sobelx64f)
        sobel_8u = np.uint8(abs_sobel64f)
        plt.imshow(sobel_8u.reshape(window_height, window_width, 3)) 
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display the denoised image
        ax = plt.subplot(m, n, i +1 + 4*n)
        _denoise = np.copy(denoise[i])
        _denoise *= 255
        _denoise = _denoise.astype('uint8')
        plt.imshow(_denoise.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display diff between the original and the denoised image
        ax = plt.subplot(m, n, i +1 + 5*n)
        kernel = np.ones((5,5),np.float32)/25
        __denoise = cv2.filter2D(_denoise,-1,kernel)
        sobelx64f = cv2.Sobel(__denoise,cv2.CV_64F,1,0,ksize=5)
        abs_sobel64f = np.absolute(sobelx64f)
        sobel_8u = np.uint8(abs_sobel64f)
        plt.imshow(sobel_8u.reshape(window_height, window_width, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

In [ ]:
model = load_model(model_name + '.' + str(testimage_epoch).zfill(3) + '.h5')
model.summary()
print("testimages: ", testimages_max)
with tf.device('/cpu:0'):
    score = model.evaluate(X_test[0:testimages_max], Y_test[0:testimages_max])
    print(type(score))
    magazine_test = model.predict(X_test[0:testimages_max])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 activation (Activation)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 64)     256       
 ormalization)                                                   
                                                                 
 activation_1 (Activation)   (None, 256, 256, 64)      0         
                                                             

In [ ]:
for i in range(int(testimages_max/5)):
    showOrigDec(Y_test[i*5:], X_test[i*5:], magazine_test[i*5:])